<a href="https://colab.research.google.com/github/shan-gao-qd/metro/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/My Drive/Dissertation')

In [ ]:
import pandas as pd
import numpy as np

load datasets,
the datasets are coverted from GeoTIFF to csv in QGIS

In [ ]:
light={}
x=0
for i in range(2000, 2019):
  light["light{0}".format(x)] = pd.read_csv(f'light{i}.csv', float_precision='round_trip', header=None, delim_whitespace=True)
  x+=1

merge the datasets to form a panel

In [ ]:
light2000 = light['light0']
light2001 = light['light1']
light2002 = light['light2']
light2003 = light['light3']
light2004 = light['light4']
light2005 = light['light5']
light2006 = light['light6']
light2007 = light['light7']
light2008 = light['light8']
light2009 = light['light9']
light2010 = light['light10']
light2011 = light['light11']
light2012 = light['light12']
light2013 = light['light13']
light2014 = light['light14']
light2015 = light['light15']
light2016 = light['light16']
light2017 = light['light17']
light2018 = light['light18']

In [ ]:
light_list = [light2000, light2001, light2002, light2003, light2004, light2005, light2006, light2007, 
              light2008, light2009, light2010, light2011, light2012, light2013, light2014, light2015,
              light2016, light2017, light2018]

In [ ]:
new_list = []
x=0
for i in light_list:
  i = i.rename(columns={0: "lon", 1: "lat", 2: "light"})
  time = [2000+x]*len(i)
  i['time'] = time
  new_list.append(i)
  x+=1

In [ ]:
lightdf = pd.concat(new_list)

In [ ]:
lightdf['index'] = lightdf['lon'].astype(str) + ',' + lightdf['lat'].astype(str)

In [ ]:
stations = pd.read_csv('stations.csv', float_precision='round_trip', sep=',')

In [ ]:
from math import cos, asin, sqrt, pi
## the Haversine formula for calculating distance
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a))

nearest distance

In [ ]:
stcoods = stations[['long', 'lati']].to_numpy()

In [ ]:
lightdf['distance1'] = range(0, len(lightdf))
for i in range(2000, 2019):
  s = stations[stations['year'] < i+1]
  stcoods = s[['long', 'lati']].to_numpy()
  dl=[]
  for index, row in lightdf[lightdf['time']==i].iterrows():
    distances = []
    for j in stcoods:
      distances.append(distance(row[3], row[2], j[1], j[0]))
    dl.append(min(distances))
  lightdf.loc[lightdf['time']==i, 'distance1'] = dl

distance bands

In [ ]:
for i in range(2000, 2019):
  s = stations[stations['year'] < i+1]
  stcoods = s[['long', 'lati']].to_numpy()
  s1=[]
  s3=[]
  s5=[]
  s10=[]
  for index, row in lightdf[lightdf['time']==i].iterrows():
    distances = []
    for j in stcoods:
      distances.append(distance(row[3], row[2], j[1], j[0]))
    s1.append(len([distance for distance in distances if distance < 1]))
    s3.append(len([distance for distance in distances if distance < 3]))
    s5.append(len([distance for distance in distances if distance < 5]))
    s10.append(len([distance for distance in distances if distance < 10]))
  lightdf.loc[lightdf['time']==i, 'num_s_1km'] = s1
  lightdf.loc[lightdf['time']==i, 'num_s_3km'] = s3
  lightdf.loc[lightdf['time']==i, 'num_s_5km'] = s5
  lightdf.loc[lightdf['time']==i, 'num_s_10km'] = s10

lag and leads

In [ ]:
for t in [1,2,3,4,5]:
  for i in range(2000, 2019):
    s = stations[stations['year'] < i+1-t]
    stcoods = s[['long', 'lati']].to_numpy()
    dl=[]
    for index, row in lightdf[lightdf['time']==i].iterrows():
      distances = []
      for j in stcoods:
        distances.append(distance(row[3], row[2], j[1], j[0]))
      dl.append(min(distances))
    lightdf.loc[lightdf['time']==i, f'distance-{t}'] = dl

In [ ]:
for t in [1,2,3,4,5]:
  for i in range(2000, 2019):
    s = stations[stations['year'] < i+1+t]
    stcoods = s[['long', 'lati']].to_numpy()
    dl=[]
    for index, row in lightdf[lightdf['time']==i].iterrows():
      distances = []
      for j in stcoods:
        distances.append(distance(row[3], row[2], j[1], j[0]))
      dl.append(min(distances))
    lightdf.loc[lightdf['time']==i, f'distance+{t}'] = dl

In [ ]:
for t in [1,2,3,4,5]:
  for i in range(2000, 2019):
    s = stations[stations['year'] < i+1-t]
    stcoods = s[['long', 'lati']].to_numpy()
    s1=[]
    s3=[]
    s5=[]
    s10=[]
    for index, row in lightdf[lightdf['time']==i].iterrows():
      distances = []
      for j in stcoods:
        distances.append(distance(row[3], row[2], j[1], j[0]))
      s1.append(len([distance for distance in distances if distance < 1]))
      s3.append(len([distance for distance in distances if distance < 3]))
      s5.append(len([distance for distance in distances if distance < 5]))
      s10.append(len([distance for distance in distances if distance < 10]))
    lightdf.loc[lightdf['time']==i, f'num_s_1km-{t}'] = s1
    lightdf.loc[lightdf['time']==i, f'num_s_3km-{t}'] = s3
    lightdf.loc[lightdf['time']==i, f'num_s_5km-{t}'] = s5
    lightdf.loc[lightdf['time']==i, f'num_s_10km-{t}'] = s10

In [ ]:
for t in [1,2,3,4,5]:
  for i in range(2000, 2019):
    s = stations[stations['year'] < i+1+t]
    stcoods = s[['long', 'lati']].to_numpy()
    s1=[]
    s3=[]
    s5=[]
    s10=[]
    for index, row in lightdf[lightdf['time']==i].iterrows():
      distances = []
      for j in stcoods:
        distances.append(distance(row[3], row[2], j[1], j[0]))
      s1.append(len([distance for distance in distances if distance < 1]))
      s3.append(len([distance for distance in distances if distance < 3]))
      s5.append(len([distance for distance in distances if distance < 5]))
      s10.append(len([distance for distance in distances if distance < 10]))
    lightdf.loc[lightdf['time']==i, f'num_s_1km+{t}'] = s1
    lightdf.loc[lightdf['time']==i, f'num_s_3km+{t}'] = s3
    lightdf.loc[lightdf['time']==i, f'num_s_5km+{t}'] = s5
    lightdf.loc[lightdf['time']==i, f'num_s_10km+{t}'] = s10

regression

In [ ]:
lightdf = lightdf.set_index(['index', 'time'])
exog = sm.tools.tools.add_constant(lightdf['distance1'])
endog = lightdf['light']
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects=True , time_effects=True) 
fe_res = model_fe.fit(cov_type='robust') 
#print results
print(fe_res)

firm establishments

In [ ]:
ged = pd.read_csv('GED_2005-2015_v2.csv', float_precision='round_trip', sep=',')
ged = ged[ged['province_code']==310000]
ged = ged[['idx', 'lon', 'lat', 'date', 'establish_', 'establis_1', 'establis_2', 'establis_3', 'exist_tota',	'exist_prim', '', 'county_cod', 'county', 'city_code', 'city']]
pd.set_option('display.precision',16)

In [ ]:
ged['lon'] = pd.to_numeric(ged["lon"])
ged['lat'] = pd.to_numeric(ged["lat"])
ged['establish_'] = pd.to_numeric(ged["establish_"])
ged['establis_1'] = pd.to_numeric(ged["establis_1"])
ged['establis_2'] = pd.to_numeric(ged["establis_2"])
ged['establis_3'] = pd.to_numeric(ged["establis_3"])

ged['exist_tota'] = pd.to_numeric(ged["exist_tota"])
ged['exist_prim'] = pd.to_numeric(ged["exist_prim"])
ged['exist_seco'] = pd.to_numeric(ged["exist_seco"])
ged['exist_tert'] = pd.to_numeric(ged["exist_tert"])
ged['date'] = pd.to_numeric(ged["date"])

In [ ]:
df_balanced = (ged.set_index('date',append=True)
                 .reindex(pd.MultiIndex.from_product([ged.idx.unique(),
                                                      range(ged.date.min(), ged.date.max()+1)],
                                                      names=['idx','date']))
                 .reset_index(level=1))


In [ ]:
import itertools

balanced_idx = pd.DataFrame(
    itertools.product(
        set(ged['idx']),  # Dimension 1: All country codes
        range(2005, 2016)
    )
    , columns=['idx', 'date']  # Assign column names
)


In [ ]:
balanced_df = balanced_idx.merge(
    ged, 
    on=['idx', 'date'], 
    how='left'
)


In [ ]:
# Fill 1: Simply filled by zero.
balanced_df['establish_'].fillna(0, inplace=True)
balanced_df['establis_1'].fillna(0, inplace=True)
balanced_df['establis_2'].fillna(0, inplace=True)
balanced_df['establis_3'].fillna(0, inplace=True)
balanced_df['exist_tota'].fillna(0, inplace=True)
balanced_df['exist_prim'].fillna(0, inplace=True) 
balanced_df['exist_seco'].fillna(0, inplace=True) 
balanced_df['exist_tert'].fillna(0, inplace=True)

In [ ]:
stcoods = stations[['long', 'lati']].to_numpy()
for i in [[121.5, 31.2]]:
  distances = []
  for j in stcoods:
    distances.append(distance(i[1], i[0], j[1], j[0]))
  print(min(distances))

In [ ]:
for t in [1,2,3,4,5]:
  for i in range(2005, 2016):
    s = stations[stations['year'] < i+1-t]
    stcoods = s[['long', 'lati']].to_numpy()
    s1=[]
    s3=[]
    s5=[]
    s10=[]
    for index, row in balanced_df[balanced_df['date']==i].iterrows():
      distances = []
      for j in stcoods:
        distances.append(distance(row[4], row[3], j[1], j[0]))
      s1.append(len([distance for distance in distances if distance < 1]))
      s3.append(len([distance for distance in distances if distance < 3]))
      s5.append(len([distance for distance in distances if distance < 5]))
      s10.append(len([distance for distance in distances if distance < 10]))
    balanced_df.loc[balanced_df['date']==i, f'num_s_1km-{t}'] = s1
    balanced_df.loc[balanced_df['date']==i, f'num_s_3km-{t}'] = s3
    balanced_df.loc[balanced_df['date']==i, f'num_s_5km-{t}'] = s5
    balanced_df.loc[balanced_df['date']==i, f'num_s_10km-{t}'] = s10

In [ ]:
for t in [1,2,3,4,5]:
  for i in range(2005, 2016):
    s = stations[stations['year'] < i+1+t]
    stcoods = s[['long', 'lati']].to_numpy()
    s1=[]
    s3=[]
    s5=[]
    s10=[]
    for index, row in balanced_df[balanced_df['date']==i].iterrows():
      distances = []
      for j in stcoods:
        distances.append(distance(row[4], row[3], j[1], j[0]))
      s1.append(len([distance for distance in distances if distance < 1]))
      s3.append(len([distance for distance in distances if distance < 3]))
      s5.append(len([distance for distance in distances if distance < 5]))
      s10.append(len([distance for distance in distances if distance < 10]))
    balanced_df.loc[balanced_df['date']==i, f'num_s_1km+{t}'] = s1
    balanced_df.loc[balanced_df['date']==i, f'num_s_3km+{t}'] = s3
    balanced_df.loc[balanced_df['date']==i, f'num_s_5km+{t}'] = s5
    balanced_df.loc[balanced_df['date']==i, f'num_s_10km+{t}'] = s10